In [6]:
import geopandas as gpd
import matplotlib.pyplot as plt
#import mplleaflet
#import folium
#from folium import plugins
import pandas as pd
#from shapely.geometry import Point, Polygon
%matplotlib inline
import numpy as np
import requests
import json

"""The column for the centroid of the census block (or the data point you're using for where people live)
must be called "centroids"

The column for the gis point location of the markets must be called geometry"""

'The column for the centroid of the census block (or the data point you\'re using for where people live)\nmust be called "centroids"\n\nThe column for the gis point location of the markets must be called geometry'

Requires: GTFS data from Allegheny County Port Authoridy, Open Street Map (OSM) data, and Open Trip Planner (OTP)

Open Trip Planner
Download otp-2.0.0-shaded.jar from https://repo1.maven.org/maven2/org/opentripplanner/otp/2.0.0-rc1/

GTFS data
Must download Port Authority GTFS data from https://www.portauthority.org/business-center/developer-resources/

OSM data
Using openstreetmap for walking, biking, driving, etc
OSM relation for Allegheny County: 2563472
OSM relation for Pittsburgh: 188553
Download osm pbf for the Allegheny county area from https://www.interline.io/osm/extracts/
You will need to register on the website to get an api token to be able to download 

Put the gtfs data (or zip file) and osm data in the same folder

In the command line, navigate to the directory where the OTP jar file is

type "java -Xmx2G -jar otp-2.0.0-rc1-shaded.jar --build --serve \<File path to the gtfs and osm data>"
(Don't include the <  > ' s in your command)

This will create an instance of the OTP server on your local machine. The -Xmx2G means it can use 2GB of RAM. 
Check that your instance of OTP is running by going to http://localhost:8080/
Do a test route here to make sure it's working with the GTFS and OSM data

In [2]:
#Shapefile data for Allegheny county census blocks located at 
#https://catalog.data.gov/dataset/tiger-line-shapefile-2019-county-allegheny-county-pa-topological-faces-polygons-with-all-geocod
#points_fp = r"geospatial_data/tl_2019_42003_faces.shp"
#points = gpd.read_file(points_fp)

In [4]:
"Block geoJSON downloaded from https://data.wprdc.org/dataset/allegheny-county-census-blocks-20161"
#blocks_file = r"geospatial_data/Allegheny_County_Census_Blocks_2016.geojson"
#blocks = gpd.read_file(blocks_file)
tract_file = r"geospatial_data/Allegheny_County_Census_Tracts_2016.geojson"
tracts = gpd.read_file(tract_file)

In [5]:
tracts

,FID,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,SHAPE_Length,SHAPE_Area,geometry
0,1,42,003,050900,1400000US42003050900,42003050900,509,CT,452643.0,4114.0,0.045048,0.000046,"MULTIPOLYGON (((-79.98390 40.44853, -79.98095 ..."
1,2,42,003,070300,1400000US42003070300,42003070300,703,CT,428546.0,0.0,0.028484,0.000046,"MULTIPOLYGON (((-79.94095 40.45132, -79.93681 ..."
2,3,42,003,120700,1400000US42003120700,42003120700,1207,CT,514927.0,0.0,0.031372,0.000055,"MULTIPOLYGON (((-79.90955 40.45670, -79.90804 ..."
3,4,42,003,140400,1400000US42003140400,42003140400,1404,CT,834995.0,0.0,0.038374,0.000089,"MULTIPOLYGON (((-79.92158 40.44768, -79.92023 ..."
4,5,42,003,180700,1400000US42003180700,42003180700,1807,CT,393380.0,0.0,0.026142,0.000042,"MULTIPOLYGON (((-80.00561 40.42702, -80.00384 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,398,42,003,141000,1400000US42003141000,42003141000,1410,CT,507878.0,0.0,0.046226,0.000055,"MULTIPOLYGON (((-79.90137 40.42724, -79.89868 ..."
398,399,42,003,202300,1400000US42003202300,42003202300,2023,CT,2532255.0,0.0,0.075588,0.000270,"MULTIPOLYGON (((-80.04760 40.40779, -80.04373 ..."
399,400,42,003,320600,1400000US42003320600,42003320600,3206,CT,1479651.0,0.0,0.053969,0.000159,"MULTIPOLYGON (((-80.01502 40.39296, -80.01398 ..."
400,401,42,003,408001,1400000US42003408001,42003408001,4080.01,CT,26224976.0,0.0,0.220845,0.002790,"MULTIPOLYGON (((-79.99418 40.67364, -79.98785 ..."


In [4]:
blocks['centroids'] = blocks['geometry'].apply(lambda x: x.centroid)

In [5]:
#Get population data from the 
pod_file = r"POD_Sites_Geo.csv"
pod_sites = pd.read_csv(pod_file)
pods = gpd.GeoDataFrame(pod_sites, geometry=gpd.points_from_xy(pod_sites.Longitude, pod_sites.Latitude))
pods.set_crs(epsg=4326, inplace=True)



,SCHOOL/FACILITY NAME,EMS AGENCY PHONE NUMBER,CLOSEST HOSPITAL(S),HOSPITAL(S) PHONE NUMBER,STRIP MAP,Latitude,Longitude,STRIP MAP TO CLOSEST HOSPITAL(S),Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,geometry
0,AVONWORTH HIGH SCHOOL,(412) 931-8200,OHIO VALLEY GENERAL\n UPMC PASSAVANT,(412) 777-6161\n (412) 367-6700,"GOOGLE MAPS: 304 JOSEPHS LN, PITTSBURGH, PA 15237",40.532038,-80.077886,AVONWORTH TO OHIO VALLEY\n \n AVONWORTH TO UPM...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-80.07789 40.53204)
1,BALDWIN HIGH SCHOOL,(412) 343-5111,JEFFERSON REGIONAL\n ST CLAIR MEMORIAL,(412) 469-5000\n (412) 942-4000,"GOOGLE MAPS: 4653 CLARITON BLVD, PITTSBURGH, P...",40.355905,-79.977319,BALDWIN TO JEFFERSON REGIONAL\n \n BALDWIN TO ...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-79.97732 40.35590)
2,BETHEL PARK SR HIGH SCHOOL,(412) 831-3710,JEFFERSON REGIONAL\n ST CLAIR MEMORIAL,(412) 469-5000\n (412) 942-4000,"GOOGLE MAPS: 309 CHURCH RD, BETHEL PARK, PA 15102",40.333245,-80.041704,BETHEL PARK TO ST CLAIR MEMORIAL\n \n BETHEL P...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-80.04170 40.33324)
3,BRASHEAR HIGH SCHOOL,(412) 622-6930,ALLEGHENY GENERAL\n UPMC MERCY,(412) 359-3252\n (412) 232-8111,"GOOGLE MAPS: 590 CRANE AVE, PITTSBURGH, PA 15216",40.418258,-80.018085,BRASHEAR TO ALLEGHENY GENERAL\n \n BRASHEAR TO...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-80.01808 40.41826)
4,BRENTWOOD JR SR HIGH SCHOOL,(412) 884-8740,JEFFERSON REGIONAL,(412) 469-5000,"GOOGLE MAPS: 3601 BROWNSVILLE RD, PITTSBURGH, ...",40.374427,-79.974117,BRENTWOOD TO JEFFERSON REGIONAL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-79.97412 40.37443)
5,CARLYNTON SENIOR HIGH,(412) 787-5044,OHIO VALLEY GENERAL\n ST CLAIR MEMORIAL,(412) 777-6161\n (412) 942-4000,"GOOGLE MAPS: 435 KINGS HWY, CARNEGIE, PA 15106",40.424500,-80.096975,CARLYNTON TO OHIO VALLEY \n \n CARLYNTON TO ST...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-80.09698 40.42450)
6,CARRICK HIGH SCHOOL,(412) 622-6930,JEFFERSON REGIONAL\n UPMC MERCY,(412) 469-5000\n (412) 232-8111,"GOOGLE MAPS: 125 PARKFIELD ST, PITTSBURGH, PA ...",40.390925,-79.988959,CARRICK TO UPMC MERCY\n \n CARRICK TO JEFFERSO...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-79.98896 40.39093)
7,CHARTIERS VALLEY SR HIGH,(412) 221-8382\n (412) 276-9652,ST CLAIR MEMORIAL,(412) 942-4000,"GOOGLE MAPS: 2030 SWALLOW HILL RD, PITTSBURGH,...",40.398360,-80.079176,CHARTIERS VALLEY TO ST CLAIR MEMORIAL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-80.07918 40.39836)
8,CLAIRTON JR SR HIGH SCHOOL,(412) 233-8113,UPMC MCKEESPORT,(412) 664-2000,"GOOGLE MAPS: 501 WADDELL AVE, CLAIRTON, PA 15025",40.294394,-79.886218,CLAIRTON TO UPMC MCKEESPORT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-79.88622 40.29439)
9,CORNELL JR/SR HIGH SCHOOL,(412) 262-2621,HERITAGE VALLEY - SEWICKLEY\n OHIO VALLEY GENERAL,(412) 741-6600\n \n (412) 777-6161,"GOOGLE MAPS: 1099 MAPLE STREET EXTENSION, CORA...",40.508473,-80.162744,CORNELL TO HERITAGE VALLEY\n \n CORNELL TO OHI...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-80.16274 40.50847)


In [6]:
blocks.head()

,FID,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYPE,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,SHAPE_Length,SHAPE_Area,geometry,centroids
0,1,42,003,523100,4022,420035231004022,Block 4022,G5040,U,69697,U,S,29894.0,0.0,+40.4626275,-079.8838755,0.008087,3.174089e-06,"MULTIPOLYGON (((-79.88579 40.46240, -79.88320 ...",POINT (-79.88388 40.46264)
1,2,42,003,415001,3015,420034150013015,Block 3015,G5040,U,69697,U,S,463508.0,0.0,+40.5706959,-079.8451995,0.030601,4.929256e-05,"MULTIPOLYGON (((-79.84875 40.56825, -79.84853 ...",POINT (-79.84520 40.57070)
2,3,42,003,483800,3014,420034838003014,Block 3014,G5040,U,69697,U,S,3403.0,0.0,+40.4024110,-079.9080700,0.002683,3.610573e-07,"MULTIPOLYGON (((-79.90863 40.40216, -79.90849 ...",POINT (-79.90807 40.40242)
3,4,42,003,429202,3017,420034292023017,Block 3017,G5040,U,69697,U,S,7721.0,0.0,+40.5147292,-080.0040423,0.004652,8.204294e-07,"MULTIPOLYGON (((-80.00437 40.51588, -80.00424 ...",POINT (-80.00405 40.51474)
4,5,42,003,508000,2010,420035080002010,Block 2010,G5040,U,69697,U,S,13223.0,0.0,+40.3918463,-079.8053084,0.006781,1.402579e-06,"MULTIPOLYGON (((-79.80667 40.39119, -79.80626 ...",POINT (-79.80531 40.39185)


In [7]:
pods.reset_index(inplace=True)

In [8]:
pods.head()

,index,SCHOOL/FACILITY NAME,EMS AGENCY PHONE NUMBER,CLOSEST HOSPITAL(S),HOSPITAL(S) PHONE NUMBER,STRIP MAP,Latitude,Longitude,STRIP MAP TO CLOSEST HOSPITAL(S),Unnamed: 8,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,geometry
0,0,AVONWORTH HIGH SCHOOL,(412) 931-8200,OHIO VALLEY GENERAL\n UPMC PASSAVANT,(412) 777-6161\n (412) 367-6700,"GOOGLE MAPS: 304 JOSEPHS LN, PITTSBURGH, PA 15237",40.532038,-80.077886,AVONWORTH TO OHIO VALLEY\n \n AVONWORTH TO UPM...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-80.07789 40.53204)
1,1,BALDWIN HIGH SCHOOL,(412) 343-5111,JEFFERSON REGIONAL\n ST CLAIR MEMORIAL,(412) 469-5000\n (412) 942-4000,"GOOGLE MAPS: 4653 CLARITON BLVD, PITTSBURGH, P...",40.355905,-79.977319,BALDWIN TO JEFFERSON REGIONAL\n \n BALDWIN TO ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-79.97732 40.35590)
2,2,BETHEL PARK SR HIGH SCHOOL,(412) 831-3710,JEFFERSON REGIONAL\n ST CLAIR MEMORIAL,(412) 469-5000\n (412) 942-4000,"GOOGLE MAPS: 309 CHURCH RD, BETHEL PARK, PA 15102",40.333245,-80.041704,BETHEL PARK TO ST CLAIR MEMORIAL\n \n BETHEL P...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-80.04170 40.33324)
3,3,BRASHEAR HIGH SCHOOL,(412) 622-6930,ALLEGHENY GENERAL\n UPMC MERCY,(412) 359-3252\n (412) 232-8111,"GOOGLE MAPS: 590 CRANE AVE, PITTSBURGH, PA 15216",40.418258,-80.018085,BRASHEAR TO ALLEGHENY GENERAL\n \n BRASHEAR TO...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-80.01808 40.41826)
4,4,BRENTWOOD JR SR HIGH SCHOOL,(412) 884-8740,JEFFERSON REGIONAL,(412) 469-5000,"GOOGLE MAPS: 3601 BROWNSVILLE RD, PITTSBURGH, ...",40.374427,-79.974117,BRENTWOOD TO JEFFERSON REGIONAL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-79.97412 40.37443)


In [9]:
"""
num_closest = 3
#closest = pd.DataFrame(columns=['closest_indices', 'closest_distances'])
def findMarketDistance(number, block):
    #n_markets = {'indi': [], 'distances': []}
    indi = []
    dists = []
    for index, row in supermarket.iterrows():
        mark = row['geometry']
        distance = block.distance(mark)
        if len(dists) < number:
            indi.append(index)
            dists.append(distance)
        else:
            max_id = np.argmax(dists)
            max_dist = dists[max_id]

            if distance < max_dist:
                dists[max_id] = distance
                indi[max_id] = index
        
    return(indi, dists)
"""
                

"\nnum_closest = 3\n#closest = pd.DataFrame(columns=['closest_indices', 'closest_distances'])\ndef findMarketDistance(number, block):\n    #n_markets = {'indi': [], 'distances': []}\n    indi = []\n    dists = []\n    for index, row in supermarket.iterrows():\n        mark = row['geometry']\n        distance = block.distance(mark)\n        if len(dists) < number:\n            indi.append(index)\n            dists.append(distance)\n        else:\n            max_id = np.argmax(dists)\n            max_dist = dists[max_id]\n\n            if distance < max_dist:\n                dists[max_id] = distance\n                indi[max_id] = index\n        \n    return(indi, dists)\n"

In [10]:
#Function to get travel plan information from OTP
#Much of this is taken from rickl4's github


def route(start, finish, date, time, method):
    #bins = [1800,3600,5400,7200,9000,10800]
    #cutoff = f'&cutoffSec={bins[0]}&cutoffSec={bins[1]}&cutoffSec={bins[2]}&cutoffSec={bins[3]}&cutoffSec={bins[4]}&cutoffSec={bins[5]}'
    #mode = 'WALK,TRANSIT'
    mode = method
    #maxwalk = maxwalk
    params = f'?toPlace={finish}&fromPlace={start}&arriveBy=FALSE&mode={mode}&date={date}&time={time}&maxWalkDistance=10000'
    url = f'http://localhost:8080/otp/routers/default/plan'+params
    response = requests.get(url)
    return(response.json())

In [11]:
#Find the fastest route from the different plans return, return it's info in json format

def fastestRoute(info):
    itins = info['plan']['itineraries']
    #print(itins)
    dur = []
    for plans in itins:
        dur.append(plans['duration'])
            
    try:
        min_index = dur.index(min(dur))
    
    #If there are no routes in the itinerary
    except:
        min_index = -1
    
    if min_index != -1:
        output = itins[min_index]
        return(output)
    
    #Make the output infinity
    else:
        output = {'duration': float("inf")}
        return(output)

In [12]:
#Find the fastest route from the different plans return, return it's info in json format

def fastestRoute(info):
    itins = info['plan']['itineraries']
    #print(itins)
    dur = []
    for plans in itins:
        dur.append(plans['duration'])
            
    try:
        min_index = dur.index(min(dur))
    
    #If there are no routes in the itinerary
    except:
        min_index = -1
    
    if min_index != -1:
        output = itins[min_index]
        return(output)
    
    #Make the output infinity
    else:
        output = {'duration': float("inf")}
        return(output)

In [13]:
def travelTime(fastest_route):
    time = fastest_route['duration']
    return(time)

In [14]:
#You will need the correct geo_market as the geometry info extracted from the market dataframe for this function

#Find the market with the shortest travel time from a list of the physically closest markets
#Inputs: 
#n_closest: Each value in the lists corresponds to the n physically closest markets to a block
#block_ponints: the column of the block datafrae containing the geographic centroid of that block
#Returns the index of the market dataframe that corresponds to the closest market, by transit time

#Input geometry of pod, geometry of block points

def findTime(pods, block_points):
    #times = []
    
    #Loop through all the pods
    for row, ind in pods.iterrows():
        end_point = ind['geometry']
        end_lon = end_point.x
        end_lat = end_point.y
        end = str(end_lat) + ',' + str(end_lon)
        
        start_point = block_points['centroids']
        start_lon = start_point.x
        start_lat = start_point.y
        start = str(start_lat) + ',' + str(start_lon)
    

        routes = route(start, end, '10-26-2021', '10:30am', 'TRANSIT,WALK')
        fastest=fastestRoute(routes)
        fast_time = travelTime(fastest)
        print('transit time ', fast_time)

        #fastest = fastestRoute(routes)
        #fast_time = travelTime(fastest)
        #transit_times['POD'] = ind['SCHOOL/FACILITY NAME']
        print(block_points['GEOID10'])
        #transit_times[block_points['GEOID10']] = [fast_time]
        #transit_times[ind['SCHOOL/FACILITY NAME']] = 
        
        routes = route(start, end, '10-26-2021', '10:30am', 'WALK')
        fastest=fastestRoute(routes)
        fast_time = travelTime(fastest)
        
        #fastest = fastestRoute(routes)
        #fast_time = travelTime(fastest)
        walk_times['POD'] = ind['SCHOOL/FACILITY NAME']
        walk_times[block_points['GEOID10']] = [fast_time]
        print('walk time: ', fast_time)
        
        
        routes = route(start, end, '10-26-2021', '10:30am', 'CAR')
        fastest=fastestRoute(routes)
        fast_time = travelTime(fastest)
        
        #fastest = fastestRoute(routes)
        #fast_time = travelTime(fastest)
        drive_times['POD'] = ind['SCHOOL/FACILITY NAME']
        drive_times[block_points['GEOID10']] = [fast_time]
        
        
        
        
    
        #times.append(fast_time)
    #closest = times.index(min(times))
    #closest_time = times[closest]
    #closest_market = n_closest[closest]
    return None

In [15]:
#You will need the correct geo_market as the geometry info extracted from the market dataframe for this function

#Find the market with the shortest travel time from a list of the physically closest markets
#Inputs: 
#n_closest: Each value in the lists corresponds to the n physically closest markets to a block
#block_ponints: the column of the block datafrae containing the geographic centroid of that block
#Returns the index of the market dataframe that corresponds to the closest market, by transit time

#Input geometry of pod, geometry of block points


def findTime(pods, block_points):
    #times = []
    
    #Loop through all the pods
    for row, ind in pods.iterrows():
        walk_pod_list = [ind['SCHOOL/FACILITY NAME']]
        transit_pod_list = [ind['SCHOOL/FACILITY NAME']]
        drive_pod_list = [ind['SCHOOL/FACILITY NAME']]
        
        for block_row, block_ind in block_points.iterrows():
            end_point = ind['geometry']
            end_lon = end_point.x
            end_lat = end_point.y
            end = str(end_lat) + ',' + str(end_lon)
        
            start_point = block_ind['centroids']
            start_lon = start_point.x
            start_lat = start_point.y
            start = str(start_lat) + ',' + str(start_lon)
    
            #print(start)
            #print(end)
            routes = route(start, end, '10-26-2021', '10:30am', 'TRANSIT,WALK')
            fastest=fastestRoute(routes)
            fast_time = travelTime(fastest)
            #print('transit time ', fast_time)
            transit_pod_list.append(fast_time)

            #fastest = fastestRoute(routes)
            #fast_time = travelTime(fastest)
            #transit_times['POD'] = ind['SCHOOL/FACILITY NAME']
            #print(block_points['GEOID10'])
            #transit_times[block_points['GEOID10']] = [fast_time]
            walk_pod_list.append(fast_time)
        
            routes = route(start, end, '10-26-2021', '10:30am', 'WALK')
            fastest=fastestRoute(routes)
            fast_time = travelTime(fastest)
        
            #fastest = fastestRoute(routes)
            #fast_time = travelTime(fastest)
            #walk_times['POD'] = ind['SCHOOL/FACILITY NAME']
            #walk_times[block_points['GEOID10']] = [fast_time]
            #print('walk time: ', fast_time)
        
        
            routes = route(start, end, '10-26-2021', '10:30am', 'CAR')
            fastest=fastestRoute(routes)
            fast_time = travelTime(fastest)
        
            #fastest = fastestRoute(routes)
            #fast_time = travelTime(fastest)
            #drive_times['POD'] = ind['SCHOOL/FACILITY NAME']
            #drive_times[block_points['GEOID10']] = [fast_time]
            #print('drive time, ', fast_time)
            drive_pod_list.append(fast_time)
            
        walk_times.append(walk_pod_list)
        transit_times.append(transit_pod_list)
        drive_times.append(drive_pod_list)
        
        print('done with a pod')
        
        
        
        
    
        #times.append(fast_time)
    #closest = times.index(min(times))
    #closest_time = times[closest]
    #closest_market = n_closest[closest]
    return None

In [16]:
#walk_times= {}
#transit_times = {}
#drive_times = {}
walk_times = []
transit_times = []
drive_times = []

In [ ]:
findTime(pods, blocks)

done with a pod


In [ ]:
walk_times

In [ ]:
transit_times

In [ ]:
drive_times

In [ ]:
columns = blocks['GEOID10']

In [ ]:
pods.head()

In [ ]:
blocks.head()

In [ ]:
blocks[['Transit_Test']]==blocks.apply